In [1]:
# Standard library imports
import os

# Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# EMA Workbench imports
import ema_workbench
from ema_workbench import (
    Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator,
    ema_logging, Constant, Scenario, HypervolumeMetric,
    GenerationalDistanceMetric, EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric, SpacingMetric, Constraint)
from ema_workbench.em_framework.optimization import (
    EpsilonProgress, to_problem, ArchiveLogger, epsilon_nondominated)
from ema_workbench.analysis import parcoords
from ema_workbench.em_framework.optimization import EpsilonProgress

# Custom problem formulation import
from problem_formulation import get_model_for_problem_formulation

In [23]:
def run_optimization(epsilon, nfe, scenarios, model):
    """
    Run optimization using the EMA Workbench.

    Parameters:
    epsilon (list): The epsilon values for the optimization.
    nfe (int): The number of function evaluations.
    scenarios (list): List of scenarios to be evaluated.
    model: The model to be used for optimization.

    Returns:
    tuple: A tuple containing results and convergences.
    """
    # Initialize lists to store results and convergence metrics
    results = []
    convergences = []
    # Use MultiprocessingEvaluator for parallel processing
    with MultiprocessingEvaluator(model) as evaluator:
        # Iterate over each scenario
        for scenario in scenarios:
            # Perform optimization three times for each scenario
            for i in range(3):
                # Define convergence metrics
                convergence_metrics = [
                    ArchiveLogger(
                        "./archives",
                        [l.name for l in model.levers],
                        [o.name for o in model.outcomes],
                        base_filename="optimization.tar.gz",
                    ),
                    EpsilonProgress(),
                ]

                # Run the optimization
                result, convergence = evaluator.optimize(
                    nfe=nfe,
                    searchover="levers",
                    epsilons=epsilon,
                    constraints=None,
                    convergence=convergence_metrics,
                    reference=scenario,
                )

                # Create result directory if it does not exist
                result_dir = "./archives"
                os.makedirs(result_dir, exist_ok=True)
                
                # Save the results and convergence metrics to CSV files
                result.to_csv(os.path.join(result_dir, f"result__scen{scenario.name}__seed{i}.csv"))
                pd.DataFrame(convergence).to_csv(os.path.join(result_dir, f"convergence__scen{scenario.name}__seed{i}.csv"))

                # Append the results and convergence metrics to the lists
                results.append(result)
                convergences.append(convergence)
    
    # Return the results and convergence metrics
    return results, convergences

In [ ]:
if not os.path.exists('archives'):
    os.makedirs('archives')
    
if __name__ == '__main__':
    # Set up logging to stderr with INFO level
    ema_logging.log_to_stderr(ema_logging.INFO)

    # Get the model and steps for problem formulation 6
    model, steps = get_model_for_problem_formulation(2)

    # Load scenarios from a CSV file
    scenarios_df = pd.read_csv("./data/subset_scenarios.csv")

    # Initialize a list to store scenarios
    scenarios = []
    
    # Iterate over each row in the scenarios dataframe to create Scenario objects
    for index in range(scenarios_df.shape[0]):
        scenario = {}
        for column in scenarios_df:
            if (column == 'scenario') or (column=='scenario'):
                continue
            # Update the scenario dictionary with column values
            scenario.update({column: scenarios_df.loc[index, column]})

        # Create an EMA Workbench Scenario object
        ema_scenario = Scenario(scenarios_df.loc[index, 'scenario'], **scenario)
        # Append the scenario to the list
        scenarios.append(ema_scenario)

    # Define epsilon values for the optimization (if statement used for testing multiple epsilon values)
    epsilon_values = [[1000000, 1000000, 1000000, 1000000, 1000000]]
    # Set the number of function evaluations
    nfe = 20000

    # Iterate over each set of epsilon values
    for eps in epsilon_values:
        # Run optimization for the given epsilon values and scenarios
        results, convergences = run_optimization(eps, nfe, scenarios, model)

        # Load results from the archive files
        all_results = load_results_from_files("./archives")

        # Define outcomes of interest
        outcomes_of_interest = [
            'Expected Annual Damage', 'Dike Investment Costs', 'RfR Investment Costs', 
            'Evacuation Costs', 'Expected Number of Deaths' ]
        # Extract the outcomes from the results
        outcomes = all_results[outcomes_of_interest]

        # Get the limits for the parallel coordinates plot
        limits = parcoords.get_limits(outcomes)
        # Create a ParallelAxes object with the limits
        axes = parcoords.ParallelAxes(limits)
        # Plot the outcomes on the parallel coordinates plot
        axes.plot(outcomes)

        # Set the title and save the plot as an image file
        plt.title(f'Parallel Coordinates Plot for Epsilon {eps}')
        plt.savefig(f'parallel_coordinates_eps_{eps}.png')
        plt.show()


[MainProcess/INFO] pool started with 8 workers



  0%|                                                | 0/20000 [00:00<?, ?it/s]


  0%|▏                                   | 100/20000 [00:22<1:13:43,  4.50it/s]


  1%|▎                                   | 200/20000 [00:44<1:14:22,  4.44it/s]


  2%|▌                                   | 300/20000 [01:07<1:13:35,  4.46it/s]


  2%|▋                                   | 400/20000 [01:29<1:13:34,  4.44it/s]


  2%|▉                                   | 500/20000 [02:00<1:22:05,  3.96it/s]


  3%|█                                   | 600/20000 [02:27<1:23:47,  3.86it/s]


  4%|█▎                                  | 700/20000 [02:54<1:25:01,  3.78it/s]


  4%|█▍                                  | 800/20000 [03:21<1:25:04,  3.76it/s]


  4%|█▌                                  | 900/20000 [03:47<1:23:39,  3.80it/s]


  5%|█▊                                 | 1000/20000 [04:13<1:23:03,  3.81it/s]


  6%|█▉                                 | 1100/2